In [41]:
import pandas as pd
import numpy as np
from data_retriever import read_tickers

### Format data

In [42]:
# read in data
tickers = read_tickers()['Symbol']
df = pd.DataFrame()

for symbol in tickers:
    try:
        adj_close = pd.read_csv(f'data/time_series_daily/{symbol}.csv', index_col=0).loc['5. adjusted close',:]
        adj_close.name = symbol
        df = df.append(adj_close)
    except:
        continue
        
df = df.transpose()

# convert index to datetime
df.index = pd.to_datetime(df.index, infer_datetime_format=True)

df

,ABM,ACN,AER,AI,AL,ALTG,AMWL,ANET,ARW,ASAN,...,WORK,WPP,XRX,YALA,YELP,YEXT,ZEN,ZEPP,ZIP,ZUO
2021-01-08,40.598779,262.446892,44.40,143.63,43.179891,10.37,27.14,306.12,104.35,31.58,...,42.35,56.44,23.312121,15.19,32.53,17.12,149.37,13.34,NaN,14.37
2021-01-11,40.558928,258.661600,45.76,135.62,44.804619,10.38,26.36,302.96,104.00,32.08,...,42.07,55.17,23.203232,17.71,32.31,16.60,152.26,13.57,NaN,14.23
2021-01-12,41.475514,256.724246,46.35,133.98,45.502355,10.51,26.29,316.28,104.77,34.98,...,42.14,55.56,22.985455,17.79,31.95,16.83,152.66,13.53,NaN,14.34
2021-01-13,41.575143,256.287099,44.74,143.14,44.037109,10.37,25.25,309.63,103.69,36.48,...,42.29,54.33,22.708283,18.10,31.40,16.75,147.11,13.69,NaN,14.08
2021-01-14,41.893955,254.213515,46.48,138.50,45.083713,10.09,28.70,313.10,104.11,37.14,...,41.92,55.43,22.807273,17.34,31.87,16.91,146.23,14.04,NaN,13.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-26,49.740000,283.950000,58.91,61.43,47.110000,14.40,12.12,338.46,121.96,36.10,...,43.13,69.37,23.750000,16.05,38.73,12.53,136.45,10.00,21.10,15.04
2021-05-27,50.010000,280.050000,59.00,63.20,47.080000,14.57,12.50,337.12,121.24,36.84,...,42.85,69.20,23.990000,15.94,39.81,12.31,137.05,10.37,20.85,15.06
2021-05-28,49.890000,282.160000,59.00,61.71,47.060000,14.66,12.45,339.38,120.33,36.78,...,44.04,69.19,23.450000,15.57,40.11,14.47,136.66,9.99,21.01,15.47
2021-06-01,50.660000,280.880000,60.11,71.76,48.200000,14.54,12.83,334.87,121.94,37.00,...,43.84,71.33,23.420000,17.88,39.93,13.68,134.62,10.43,23.05,14.98


### Analysis & Metrics

In [43]:
# log of percent change
pct_change = df.pct_change().apply(lambda x : np.log(1+x))

# variance
variance = pct_change.var()

# volatility (monthly)
volatility = np.sqrt(variance * 250/12) # there are 250 trading days / year

# expected daily returns
daily_expected_returns = pct_change.mean()

# expected monthly returns
monthly_expected_returns = df.resample('m').last().pct_change().mean()

In [44]:
# covariance matrix
cov_matrix = pct_change.cov()
cov_matrix

,ABM,ACN,AER,AI,AL,ALTG,AMWL,ANET,ARW,ASAN,...,WORK,WPP,XRX,YALA,YELP,YEXT,ZEN,ZEPP,ZIP,ZUO
ABM,0.000399,0.000104,0.000216,0.000119,0.000196,0.000259,-0.000011,0.000057,0.000159,0.000006,...,4.657599e-07,0.000135,0.000184,-0.000045,0.000238,0.000034,-0.000008,-0.000047,0.000243,0.000043
ACN,0.000104,0.000157,0.000096,0.000037,0.000086,0.000092,-0.000103,0.000102,0.000110,0.000099,...,4.406658e-05,0.000083,0.000064,0.000142,0.000065,0.000160,0.000097,0.000032,0.000045,0.000128
AER,0.000216,0.000096,0.000865,0.000046,0.000624,0.000227,-0.000128,0.000079,0.000183,-0.000021,...,-1.030174e-05,0.000200,0.000272,0.000229,0.000234,-0.000012,0.000073,0.000058,0.000352,0.000083
AI,0.000119,0.000037,0.000046,0.003062,0.000128,0.000442,0.001110,0.000207,0.000302,0.000742,...,9.165089e-05,0.000123,0.000189,0.001894,0.000487,0.000886,0.000373,0.000859,0.002965,0.000530
AL,0.000196,0.000086,0.000624,0.000128,0.000670,0.000219,-0.000155,0.000073,0.000150,0.000056,...,-3.919406e-06,0.000163,0.000216,0.000223,0.000188,0.000030,0.000108,0.000046,0.000501,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YEXT,0.000034,0.000160,-0.000012,0.000886,0.000030,0.000353,0.000650,0.000334,0.000220,0.000672,...,1.736614e-04,0.000088,0.000083,0.000917,0.000314,0.001456,0.000518,0.000737,-0.002105,0.000718
ZEN,-0.000008,0.000097,0.000073,0.000373,0.000108,0.000240,0.000149,0.000209,0.000110,0.000554,...,1.028553e-04,0.000049,0.000082,0.000736,0.000009,0.000518,0.000611,0.000392,-0.000355,0.000454
ZEPP,-0.000047,0.000032,0.000058,0.000859,0.000046,0.000409,0.001089,0.000243,0.000243,0.000866,...,1.373274e-04,0.000097,0.000115,0.001289,0.000243,0.000737,0.000392,0.001851,0.000655,0.000519
ZIP,0.000243,0.000045,0.000352,0.002965,0.000501,-0.000406,0.000234,-0.000275,0.000415,-0.000150,...,-2.108014e-04,0.000668,0.000020,0.003173,-0.000543,-0.002105,-0.000355,0.000655,0.002063,-0.000881


In [22]:
# correlation matrix
corr_matrix = pct_change.corr()
corr_matrix

,ABM,ACN,AER,AI,AL,ALTG,AMWL,ANET,ARW,ASAN,...,SPGI,SQ,SQNS,SQSP,STEM,STM,SWCH,SWI,SYX,TBI
ABM,1.000000,0.417684,0.366971,0.107681,0.378857,0.417278,-0.010286,0.157471,0.480503,0.008903,...,0.200139,0.066818,0.025988,-0.102554,0.213061,0.120100,0.007933,0.057224,0.181722,0.389939
ACN,0.417684,1.000000,0.259671,0.052893,0.264665,0.236377,-0.149506,0.452448,0.532682,0.216297,...,0.582548,0.375629,0.254422,0.405915,0.255546,0.454337,0.235752,0.038909,0.213880,0.143647
AER,0.366971,0.259671,1.000000,0.028095,0.819028,0.248594,-0.078644,0.149282,0.375794,-0.019228,...,0.203842,0.041392,0.150588,0.014427,0.338695,0.223684,-0.091672,-0.078499,0.010022,0.329487
AI,0.107681,0.052893,0.028095,1.000000,0.089292,0.256871,0.363647,0.207622,0.329663,0.368794,...,0.061388,0.249565,0.353443,-0.193311,0.253552,0.260398,0.488605,0.263541,0.094987,0.100145
AL,0.378857,0.264665,0.819028,0.089292,1.000000,0.272484,-0.108724,0.156256,0.349264,0.059256,...,0.151945,-0.008606,0.131542,-0.107825,0.342321,0.183553,-0.094084,-0.056646,0.029921,0.284634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
STM,0.120100,0.454337,0.223684,0.260398,0.183553,0.377356,-0.013994,0.586244,0.587529,0.470371,...,0.362868,0.541297,0.480154,0.532366,0.391260,1.000000,0.339067,0.133877,0.097603,0.013083
SWCH,0.007933,0.235752,-0.091672,0.488605,-0.094084,0.071837,0.144135,0.338133,0.238328,0.436993,...,0.276634,0.329716,0.342856,-0.068886,0.204605,0.339067,1.000000,0.182171,0.164052,0.013995
SWI,0.057224,0.038909,-0.078499,0.263541,-0.056646,0.185030,0.458495,0.361714,0.268063,0.195959,...,-0.042386,0.200546,0.208334,0.703792,-0.061308,0.133877,0.182171,1.000000,0.010852,0.095631
SYX,0.181722,0.213880,0.010022,0.094987,0.029921,0.144461,-0.024725,0.110040,0.136484,0.158685,...,0.121104,0.220147,-0.011323,-0.044953,0.256318,0.097603,0.164052,0.010852,1.000000,0.154312


### Efficient Frontier

In [ ]:
# expected total returns (with weights)

